
输入是[24,24,3]

conv1:[24,24,64](filter:[5,5,3],size=64,padding=same,strides=[1,1],lambd=0)
max_pool1:[12,12,64](ksize=[3,3],strides=[2,2])
#norm1:[12,12,64]


conv2:[12,12,64](filter:[5,5,64],size=64,padding=same,strides=[1,1])
#norm2:[12,12,64]
max_pool2:[6,6,64](ksize=[3,3],strides=[2,2])

flat:
    w_fc1
fc1:384
    w_fc2
fc2:192
    w_score
logits:10

1，数据准备

In [1]:
import tensorflow as tf
import cifar10,cifar10_input
import numpy as np
import time

batch_size = 128
epoch = 3000
data_dir = "./cifar10_data/cifar-10-batches-bin"
#下载并解压数据集
#cifar10.maybe_download_and_extract()
#在data_dir路径下分批取出batch_size的训练数据
images_train,labels_train = cifar10_input.distorted_inputs(data_dir=data_dir,batch_size=batch_size)
#在data_dir路径下生成取出测试数据
images_test,labels_test = cifar10_input.inputs(eval_data=True,data_dir=data_dir,batch_size=batch_size)

Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.


2,准备好placeholder

In [7]:

image_holder = tf.placeholder(tf.float32,[batch_size,24,24,3],name="x")

y_ = tf.placeholder(tf.int32,[batch_size],name="y")

3,初始化参数/权重

In [8]:
def variable_with_weight_loss(shape,stddev,lambd):
    weight = tf.Variable(tf.truncated_normal(shape,stddev=stddev))
    if lambd is not None:
        weight_loss = tf.multiply(tf.nn.l2_loss(weight),lambd,name="weight_loss")
        tf.add_to_collection('losses',weight_loss)
    return weight

w_conv1 = variable_with_weight_loss(shape=[5,5,3,64],stddev=5e-2,lambd=0.0)
b_conv1 = tf.Variable(tf.zeros([64]))

w_conv2 = variable_with_weight_loss(shape=[5,5,64,64],stddev=5e-2,lambd=0.0)
b_conv2 = tf.Variable(tf.zeros([64]))



4,拿到每个类别的score

In [9]:
h_conv1 = tf.nn.relu(tf.nn.conv2d(image_holder,w_conv1,strides=[1,1,1,1],padding="SAME") + b_conv1)
h_pool1 = tf.nn.max_pool(h_conv1,ksize=[1,3,3,1],strides=[1,2,2,1],padding="SAME")

h_conv2 = tf.nn.relu(tf.nn.conv2d(h_conv1,w_conv2,strides=[1,1,1,1],padding="SAME") + b_conv2)
h_pool2 = tf.nn.max_pool(h_conv2,ksize=[1,3,3,1],strides=[1,2,2,1],padding="SAME")

h_pool2_flat = tf.reshape(h_pool2,[batch_size,-1])

dim = h_pool2_flat.get_shape()[1].value
w_fc1 = variable_with_weight_loss(shape=[dim,384],stddev=0.04,lambd=0.004)
b_fc1 = tf.Variable(tf.zeros([384]))

h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,w_fc1) + b_fc1)

w_fc2 = variable_with_weight_loss(shape=[384,192],stddev=0.04,lambd=0.004)
b_fc2 = tf.Variable(tf.zeros([192]))

h_fc2 = tf.nn.relu(tf.matmul(h_fc1,w_fc2) + b_fc2)

w_score = variable_with_weight_loss(shape=[192,10],stddev=1/192.0,lambd=0.0)
b_score = tf.Variable(tf.zeros([10]))

logits = tf.matmul(h_fc2,w_score) + b_score


5,计算多分类的softmax的loss function

In [10]:
#交叉熵损失
entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logits, labels = y_,name="loss")
#求平均值
loss = tf.reduce_mean(entropy)
tf.add_to_collection('losses',loss)
loss = tf.add_n(tf.get_collection("losses"),name="total_loss")

6,准备好optimizer

In [11]:
learning_rate = 1e-3
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

7,在session里执行graph里定义的运算

In [ ]:
f = open("./_cifar10_cnn.log",'a') 
n_epoch = epoch
init = tf.global_variables_initializer()
with tf.Session() as sess:
    tf.train.start_queue_runners()
    writer = tf.summary.FileWriter("./graphs/cifar10",sess.graph)
    sess.run(init)
    for step in range(n_epoch):
        image_batch,label_batch = sess.run([images_train,labels_train])
        _,loss_value = sess.run([optimizer,loss],feed_dict={image_holder:image_batch,y_:label_batch})
        print("batch: {0} loss: {1}".format(step,loss_value))
        f.write("batch: {0} loss: {1}".format(step,loss_value))
        f.write("\n")
        
    print("optimizer finished")
    f.write("optimizer finished")
    f.write("\n")
    
    #测试模型
  
    top_k_op = tf.nn.in_top_k(logits,y_,1)
    num_examples = 10000
    import math
    num_iter = int(match.ceil(num_examples/batch_size))
    true_count = 0
    total_sample_count = num_iter*batch_size
    step=0
    while step < num_iter:
        image_batch,label_batch = sess.run([images_test,labels_test])
        pre = sess.run([top_k_op],feed_dict={image_holder:image_batch,y_:label_batch})
        true_count += np.sum(pre)
        step += 1
    pre_avrage = true_count/total_sample_count
    print("accracy: {0}".format(pre_avrage))
    f.write("accracy: {0}".format(pre_avrage))
    f.write("\n")
    f.close()
    writer.close()
   
        
    

batch: 0 loss: 20.17603302001953
batch: 1 loss: 19.367431640625
batch: 2 loss: 18.576431274414062
batch: 3 loss: 17.80093002319336
batch: 4 loss: 17.087751388549805
batch: 5 loss: 16.417638778686523
batch: 6 loss: 15.652765274047852
batch: 7 loss: 15.026819229125977
batch: 8 loss: 14.53170394897461
batch: 9 loss: 13.86192798614502
batch: 10 loss: 13.358083724975586
batch: 11 loss: 12.760099411010742
batch: 12 loss: 12.242015838623047
batch: 13 loss: 11.68750286102295
batch: 14 loss: 11.272640228271484
batch: 15 loss: 10.939799308776855
batch: 16 loss: 10.579182624816895
batch: 17 loss: 10.189455032348633
batch: 18 loss: 9.682180404663086
batch: 19 loss: 9.30709171295166
batch: 20 loss: 8.941240310668945
batch: 21 loss: 8.67982292175293
batch: 22 loss: 8.224300384521484
batch: 23 loss: 7.978802680969238
batch: 24 loss: 7.807090759277344
batch: 25 loss: 7.390982627868652
batch: 26 loss: 7.0941901206970215
batch: 27 loss: 6.911914825439453
batch: 28 loss: 6.632338523864746
batch: 29 loss:

batch: 234 loss: 1.8413975238800049
batch: 235 loss: 1.6564396619796753
batch: 236 loss: 1.83003568649292
batch: 237 loss: 1.8226819038391113
batch: 238 loss: 1.7417874336242676
batch: 239 loss: 1.7161099910736084
batch: 240 loss: 1.6571096181869507
batch: 241 loss: 1.8155174255371094
batch: 242 loss: 1.7383010387420654
batch: 243 loss: 1.6058412790298462
batch: 244 loss: 1.6264851093292236
batch: 245 loss: 1.7932682037353516
batch: 246 loss: 1.7633382081985474
batch: 247 loss: 1.6924229860305786
batch: 248 loss: 1.586107850074768
batch: 249 loss: 1.841901183128357
batch: 250 loss: 1.6939759254455566
batch: 251 loss: 1.8210827112197876
batch: 252 loss: 1.7470489740371704
batch: 253 loss: 1.8185628652572632
batch: 254 loss: 1.6998122930526733
batch: 255 loss: 1.9413414001464844
batch: 256 loss: 1.8147732019424438
batch: 257 loss: 1.8738059997558594
batch: 258 loss: 1.7190951108932495
batch: 259 loss: 1.7101304531097412
batch: 260 loss: 1.8186043500900269
batch: 261 loss: 1.8865449428558

batch: 463 loss: 1.6296470165252686
batch: 464 loss: 1.4841922521591187
batch: 465 loss: 1.5310322046279907
batch: 466 loss: 1.5946239233016968
batch: 467 loss: 1.5329930782318115
batch: 468 loss: 1.7170560359954834
batch: 469 loss: 1.6003705263137817
batch: 470 loss: 1.5847256183624268
batch: 471 loss: 1.622113823890686
batch: 472 loss: 1.4226974248886108
batch: 473 loss: 1.7246901988983154
batch: 474 loss: 1.5835545063018799
batch: 475 loss: 1.8301239013671875
batch: 476 loss: 1.545778751373291
batch: 477 loss: 1.8017330169677734
batch: 478 loss: 1.6056160926818848
batch: 479 loss: 1.620112419128418
batch: 480 loss: 1.4903028011322021
batch: 481 loss: 1.5614557266235352
batch: 482 loss: 1.6855207681655884
batch: 483 loss: 1.539716362953186
batch: 484 loss: 1.5196537971496582
batch: 485 loss: 1.546643853187561
batch: 486 loss: 1.7413477897644043
batch: 487 loss: 1.5508308410644531
batch: 488 loss: 1.3606560230255127
batch: 489 loss: 1.7524982690811157
batch: 490 loss: 1.57702147960662

batch: 692 loss: 1.5887483358383179
batch: 693 loss: 1.5108063220977783
batch: 694 loss: 1.4853379726409912
batch: 695 loss: 1.4918365478515625
batch: 696 loss: 1.4075357913970947
batch: 697 loss: 1.499424695968628
batch: 698 loss: 1.3382207155227661
batch: 699 loss: 1.5296800136566162
batch: 700 loss: 1.59912109375
batch: 701 loss: 1.5875296592712402
batch: 702 loss: 1.5580453872680664
batch: 703 loss: 1.5948959589004517
batch: 704 loss: 1.4713236093521118
batch: 705 loss: 1.6038488149642944
batch: 706 loss: 1.4878661632537842
batch: 707 loss: 1.5405949354171753
batch: 708 loss: 1.5652107000350952
batch: 709 loss: 1.4030143022537231
batch: 710 loss: 1.5743422508239746
batch: 711 loss: 1.5732918977737427
batch: 712 loss: 1.5674397945404053
batch: 713 loss: 1.5326143503189087
batch: 714 loss: 1.4193871021270752
batch: 715 loss: 1.5051056146621704
batch: 716 loss: 1.488375186920166
batch: 717 loss: 1.3489508628845215
batch: 718 loss: 1.4160475730895996
batch: 719 loss: 1.621983528137207


batch: 922 loss: 1.3013238906860352
batch: 923 loss: 1.5126008987426758
batch: 924 loss: 1.5326192378997803
batch: 925 loss: 1.5327996015548706
batch: 926 loss: 1.308176875114441
batch: 927 loss: 1.424000859260559
batch: 928 loss: 1.3948500156402588
batch: 929 loss: 1.4289954900741577
batch: 930 loss: 1.4064608812332153
batch: 931 loss: 1.64668869972229
batch: 932 loss: 1.4102280139923096
batch: 933 loss: 1.45430326461792
batch: 934 loss: 1.4368939399719238
batch: 935 loss: 1.4146747589111328
batch: 936 loss: 1.4241747856140137
batch: 937 loss: 1.3596975803375244
batch: 938 loss: 1.4374370574951172
batch: 939 loss: 1.6585144996643066
batch: 940 loss: 1.4785914421081543
batch: 941 loss: 1.5779478549957275
batch: 942 loss: 1.6825058460235596
batch: 943 loss: 1.2985413074493408
batch: 944 loss: 1.3762263059616089
batch: 945 loss: 1.2789950370788574
batch: 946 loss: 1.3503968715667725
batch: 947 loss: 1.446842908859253
batch: 948 loss: 1.4212195873260498
batch: 949 loss: 1.508620262145996


batch: 1147 loss: 1.421688437461853
batch: 1148 loss: 1.498097538948059
batch: 1149 loss: 1.5180776119232178
batch: 1150 loss: 1.3100042343139648
batch: 1151 loss: 1.2252395153045654
batch: 1152 loss: 1.3349628448486328
batch: 1153 loss: 1.4160759449005127
batch: 1154 loss: 1.4013762474060059
batch: 1155 loss: 1.2276513576507568
batch: 1156 loss: 1.3975234031677246
batch: 1157 loss: 1.349096417427063
batch: 1158 loss: 1.3523156642913818
batch: 1159 loss: 1.3205373287200928
batch: 1160 loss: 1.3386040925979614
batch: 1161 loss: 1.4198421239852905
batch: 1162 loss: 1.429045557975769
batch: 1163 loss: 1.3452585935592651
batch: 1164 loss: 1.4707612991333008
batch: 1165 loss: 1.451441764831543
batch: 1166 loss: 1.305570363998413
batch: 1167 loss: 1.4362907409667969
batch: 1168 loss: 1.321254014968872
batch: 1169 loss: 1.441792607307434
batch: 1170 loss: 1.465566873550415
batch: 1171 loss: 1.2504479885101318
batch: 1172 loss: 1.5173866748809814
batch: 1173 loss: 1.3527168035507202
batch: 117